In [1]:
%pdb on

Automatic pdb calling has been turned ON


In [1]:
# counting the words in a feed
import feedparser
import re

def getwordcounts(url):
    d = feedparser.parse(url)
    wc = {}
    
    for e in d.entries:
        if 'summary' in e: summary = e.summary
        else: summary = e.description
    
        words = getwords(e.title+' '+summary)
        for word in words:
            wc.setdefault(word, 0)
            wc[word] += 1
    return d.feed.title, wc

def getwords(html):
    txt = re.compile(r'<[^>]+>').sub('', html)
    
    words = re.compile(r'[^A-Z^a-z]+').split(txt)
    
    return [word.lower() for word in words if word != '']

apcount = {}
wordcounts = {}
feedlist = [line for line in open('feedlist.txt')]
for i, feedurl in enumerate(feedlist):
    print(i, feedurl)
    try:
        title, wc = getwordcounts(feedurl)
    except:
        continue
    wordcounts[title] = wc
    for word, count in wc.items():
        apcount.setdefault(word, 0)
        if count>1:
            apcount[word] += 1


Automatic pdb calling has been turned ON
0 http://feeds.feedburner.com/37signals/beMH

1 http://feeds.feedburner.com/blogspot/bRuz

2 http://battellemedia.com/index.xml

3 http://blog.guykawasaki.com/index.rdf

4 http://blog.outer-court.com/rss.xml

5 http://feeds.searchenginewatch.com/sewblog

6 http://blog.topix.net/index.rdf

7 http://blogs.abcnews.com/theblotter/index.rdf

8 http://feeds.feedburner.com/ConsumingExperienceFull

9 http://flagrantdisregard.com/index.php/feed/

10 http://featured.gigaom.com/feed/

11 http://gizmodo.com/index.xml

12 http://gofugyourself.typepad.com/go_fug_yourself/index.rdf

13 http://googleblog.blogspot.com/rss.xml

14 http://feeds.feedburner.com/GoogleOperatingSystem

15 http://headrush.typepad.com/creating_passionate_users/index.rdf

16 http://feeds.feedburner.com/instapundit/main

17 http://jeremy.zawodny.com/blog/rss2.xml

18 http://joi.ito.com/index.rdf

19 http://feeds.feedburner.com/Mashable

20 http://michellemalkin.com/index.rdf

21 http://mo

In [2]:
wordlist = []
for w, bc in apcount.items():
    frac = float(bc) / len(feedlist)
    if frac > 0.1 and frac < 0.5:
        wordlist.append(w)

with open('blogdata.txt', 'w') as out:
    out.write('Blog')
    for word in wordlist:
        out.write('\t{0}'.format(word))
    out.write('\n')
    for blog, wc in wordcounts.items():
        out.write(blog)
        for word in wordlist:
            if word in wc: out.write('\t{0}'.format(wc[word]))
            else: out.write('\t0')
        out.write('\n')

In [5]:
import pandas as pd
df = pd.DataFrame(wordcounts)
df = df.loc[df.index.isin(wordlist)]
df.to_csv('blogdata.csv')

In [6]:
# Hierarchical Clustering

In [1]:
def readfile(filename):
    with open(filename, 'r') as f:
        lines = [line for line in f]
    
    colnames = lines[0].strip().split('\t')[1:]
    rownames = []
    data = []
    for line in lines[1:]:
        p = line.strip().split('\t')
        rownames.append(p[0])
        data.append([float(x) for x in p[1:]])
    
    return rownames, colnames, data

In [2]:
from math import sqrt

def pearson(v1, v2):
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    sum1Sq = sum([pow(v, 2) for v in v1])
    sum2Sq = sum([pow(v, 2) for v in v2])
    
    pSum = sum([v1[i] * v2[i] for i in range(len(v1))])
    
    num = pSum - (sum1 * sum2 / len(v1))
    den = sqrt((sum1Sq - pow(sum1, 2)/len(v1)) * (sum1Sq - pow(sum1, 2)/len(v1)))
    
    if den == 0: return 0
    
    return 1.0 - num/den

In [3]:
class bicluster:
    def __init__(self, vec, left=None, right=None,
                 distance=0.0, id=None):
        self.left = left
        self.right = right
        self.vec = vec
        self.id = id
        self.distance = distance

In [4]:
def hcluster(rows, distance=pearson):
    distances = {}
    currentclustid = -1
    
    clust = [bicluster(rows[i], id=i)
             for i in range(len(rows))]
    
    while len(clust) > 1:
        lowestpair = (0, 1)
        clostest = distance(clust[0].vec, clust[1].vec)
        
        for i in range(len(clust)):
            for j in range(i+1, len(clust)):
                if (clust[i].id, clust[j].id) not in distances:
                    distances[(clust[i].id, clust[j].id)] = \
                        distance(clust[i].vec, clust[j].vec)
                    
                d = distances[(clust[i].id, clust[j].id)]
                
                if d < clostest:
                    clostest = d
                    lowestpair = (i, j)
        
        mergevec = [
            (clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i])/2.0
            for i in range(len(clust[0].vec))
        ]
        
        newcluster = bicluster(mergevec,
                               left=clust[lowestpair[0]],
                               right=clust[lowestpair[1]],
                               distance=clostest,
                               id=currentclustid
                              )
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)
        
    return clust[0]

In [5]:
blognames, words, data = readfile('blogdata.txt')
clust = hcluster(data)

In [6]:
def printclust(clust, labels=None, n=0):
    
    for i in range(n): print(' ', end='')
    if clust.id < 0:
        print('-')
    else:
        if labels == None: print(clust.id)
        else: print(labels[clust.id])
    
    if clust.left != None:
        printclust(clust.left, labels=labels, n=n+1)
    if clust.right != None:
        printclust(clust.right, labels=labels, n=n+1)

In [7]:
printclust(clust, blognames)

-
 Kotaku
 -
  Schneier on Security
  -
   Derek Powazek
   -
    gapingvoid: "cartoons drawn on the back of business cards"
    -
     -
      -
       -
        -
         Power Line
         NewsBusters.org - Exposing Liberal Media Bias
        -
         Think Progress
         -
          Talking Points Memo: by Joshua Micah Marshall
          Daily Kos
       -
        Hot Air
        SpikedHumor
      -
       -
        PerezHilton.com
        Jeremy Zawodny's blog
       -
        -
         -
          -
           -
            -
             -
              -
               -
                -
                 Wired News: Top Stories
                 -
                  GigaOM
                  Lifehacker
                -
                 -
                  TechCrunch
                  -
                   Matt Cutts: Gadgets, Google, and SEO
                   Read/WriteWeb
                 -
                  -
                   Shoemoney - Skills to pay the bills
     

In [8]:
# Drawing the Dendrogram
from PIL import Image, ImageDraw

In [9]:
def getheight(clust):
    if clust.left == None and clust.right == None:
        return 1
    return getheight(clust.right) + getheight(clust.left)

def getdepth(clust):
    if clust.left == None and clust.right == None:
        return 0
    return max(getdepth(clust.right), getdepth(clust.left)) + clust.distance

In [10]:
def drawdendrogram(clust, labels, jpeg='clusters.jpg'):
    h = getheight(clust) * 20
    w = 1200
    depth = getdepth(clust)
    
    scaling = float(w-150) / depth
    
    img = Image.new('RGB', (w,h), (255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    draw.line((0, h/2, 10, h/2), fill=(255, 0, 0))
    drawnode(draw, clust, 10, (h/2), scaling, labels)
    img.save(jpeg, 'JPEG')
    
def drawnode(draw, clust, x, y, scaling, labels):
    if clust.id >= 0:
        draw.text((x+5, y-7), labels[clust.id], (0,0,0))
        return
    
    h1 = getheight(clust.left) * 20
    h2 = getheight(clust.right) * 20
    top = y - (h1+h2)/2
    bottom = y + (h1+h2)/2
    
    ll = clust.distance * scaling
    
    draw.line((x, top+h1/2, x, bottom-h2/2), fill=(255,0,0))
    draw.line((x, top+h1/2, x+11, top+h1/2), fill=(255,0,0))
    draw.line((x, bottom-h2/2, x+11, bottom-h2/2), fill=(255,0,0))
    
    drawnode(draw, clust.left, x+11, top+h1/2, scaling, labels)
    drawnode(draw, clust.right, x+11, bottom-h2/2, scaling, labels)

In [11]:
drawdendrogram(clust, blognames, jpeg='blogclust.jpg')

In [12]:
# Column Clustering

def rotatematrix(data):
    newdata = []
    for i in range(len(data[0])):
        newrow = [data[j][i] for j in range(len(data))]
        newdata.append(newrow)
    return newdata

In [13]:
rdata = rotatematrix(data)
wordclust = hcluster(rdata)
drawdendrogram(wordclust, labels=words, jpeg='wordclust.jpg')

In [14]:
# K-Means Clustering

import random

In [15]:
def kcluster(rows, distance=pearson, k=4):
    ranges = [(min([row[i] for row in rows]), max([row[i] for row in rows]))
                 for i in range(len(rows[0]))]
    
    # k random center
    clusters = [[random.random() * (ranges[i][1]-ranges[i][0]) + ranges[i][0]
                for i in range(len(rows[0]))] for j in range(k)]
    
    lastmatches = None
    for t in range(100):
        print('Iteration {0}'.format(t))
        bestmatches = [[] for i in range(k)]
        
        for j in range(len(rows)):
            row = rows[j]
            bestmatch = 0
            for i in range(k):
                d = distance(clusters[i], row)
                if d < distance(clusters[bestmatch], row):
                    bestmatch = i
            bestmatches[bestmatch].append(j)
        
        if bestmatches == lastmatches:
            break
        lastmatches = bestmatches
        
        # move the center to the group average point
        for i in range(k):
            avgs = [0.0] * len(rows[0])
            if len(bestmatches[i]) > 0:
                for rowid in bestmatches[i]:
                    for m in range(len(avgs)):
                        avgs[m] += rows[rowid][m]
                for j in range(len(avgs)):
                    avgs[j] /= len(bestmatches[i])
                clusters[i] = avgs
    
    return bestmatches

In [16]:
kclust = kcluster(data, k=10)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [17]:
[blognames[r] for r in kclust[3]]

[]

In [21]:
# Tanimoto coefficient

def tanimoto(v1, v2):
    
    c1 = sum(v1)
    c2 = sum(v2)
    shr = sum([i1 and i2 for i1, i2 in zip(v1, v2)])
    
    return 1 - (shr/(c1 + c2 - shr))

In [23]:
wants, people, data = readfile('zebo.txt')
wclust = hcluster(data, distance=tanimoto)
drawdendrogram(wclust, wants, jpeg='wants.jpg')

In [60]:
def scaledown(data, distance=pearson, rate=0.000001):
    n = len(data)
    
    # real distance
    realdist = [[distance(data[i], data[j]) for j in range(n)]
               for i in range(n)]
    outersum = 0.0
    
    # random initial loc
    loc = [[random.random(), random.random()] for i in range(n)]
    fakedist = [[0.0 for j in range(n)] for i in range(n)]
    
    lasterror = None
    for m in range(1000):
#         fakedist = [[distance(loc[i], loc[j]) for j in range(n)]
#                    for i in range(n)]
        for i in range(n):
            for j in range(n):
                fakedist[i][j] = sqrt(sum([pow(loc[i][x] - loc[j][x], 2)
                                          for x in range(len(loc[i]))]))
        
        # move the loc
        grad = [[0.0, 0.0] for i in range(n)]
        
        totalerror = 0
        for k in range(n):
            for j in range(n):
                if j == k: continue
                errorterm = (fakedist[j][k] - realdist[j][k]) / realdist[j][k]
                
                grad[k][0] += ((loc[k][0] - loc[j][0])/fakedist[j][k]) * errorterm
                grad[k][1] += ((loc[k][1] - loc[j][1])/fakedist[j][k]) * errorterm
#                 grad[k][0] += (loc[k][0] - loc[j][0]) * errorterm
#                 grad[k][1] += (loc[k][1] - loc[j][1]) * errorterm
                
                totalerror += abs(errorterm)
        print(totalerror)
        
        if lasterror and lasterror < totalerror: break
        lasterror = totalerror
        
        # TODO: how to use matric to simplify
        for k in range(n):
            loc[k][0] -= rate * grad[k][0]
            loc[k][1] -= rate * grad[k][1]
    
    return loc

In [45]:
def draw2d(data, labels, jpeg='mds2d.jpg'):
    img = Image.new('RGB', (2000,2000), (255,255,255))
    draw = ImageDraw.Draw(img)
    for i in range(len(data)):
        x = (data[i][0] + 0.5) * 1000
        y = (data[i][1] + 0.5) * 1000
        draw.text((x,y), labels[i], (0,0,0))
    img.save(jpeg, 'JPEG')

In [61]:
blognames, words, data = readfile('blogdata.txt')
coords = scaledown(data)
draw2d(coords, blognames, jpeg='blogs2d.jpg')

7257.270927227234
7257.9170936138125


In [57]:
scaledown([[2,1], [1,2], [3,1]])

4.452014601910929
4.4180703942826804
4.383644441201949
4.34874059357193
4.313363149833152
4.2775168584904515
4.241206919700445
4.204438985894383
4.167219161415186
4.129554001151564
4.0914505081565675
4.052916130242429
4.0139587555483125
3.974586707082471
3.934808736245241
3.894634015344341
3.854072129118964
3.813133065294149
3.7718272041918692
3.7301653074300374
3.688158505745364
3.6458182859803765
3.6031564772791964
3.5689799876889663
3.5681127474588648
3.5674791230927005
3.5670828957518346
3.566927448080561
3.5670157463540937


[[0.7674815956855695, 0.7133326500178413],
 [-0.05655309807705622, 0.5758170530068707],
 [1.1904342383411575, 0.3517107416774642]]